<a href="https://colab.research.google.com/github/jiin124/Dacon/blob/main/%EC%87%BC%ED%95%91%EB%AA%B0_%EB%A6%AC%EB%B7%B0/%EC%87%BC%ED%95%91%EB%AA%B0_%EB%A6%AC%EB%B7%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

- 토픽 모델링
- data augmentation
- noise 제거
- kobert
- GPT

사용해보기


In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (6,681 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [3]:
#한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager,rc

font_path="/content/drive/MyDrive/NanumSquareB.ttf"
font=font_manager.FontProperties(fname=font_path)

In [4]:
plt.rc('font',family='NanumBarunGothic')

In [59]:
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
sample=pd.read_csv('/content/sample_submission.csv')

In [60]:
train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [61]:
test.head()

,id,reviews
0,0,채소가 약간 시들어 있어요
1,1,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2,2,부들부들 좋네요 입어보고 시원하면 또 살게요
3,3,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4,4,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요


In [62]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       25000 non-null  int64 
 1   reviews  25000 non-null  object
 2   target   25000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 586.1+ KB


In [63]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       25000 non-null  int64 
 1   reviews  25000 non-null  object
dtypes: int64(1), object(1)
memory usage: 390.8+ KB


In [64]:
train.drop('id',axis=1,inplace=True)

In [65]:
test.drop('id',axis=1,inplace=True)

In [6]:
pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 5.2 MB/s 


In [7]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 101 kB 9.9 MB/s 
     |████████████████████████████████| 596 kB 45.8 MB/s 
     |████████████████████████████████| 6.6 MB 43.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [111]:
import datasets
import tensorflow as tf
from transformers import AutoTokenizer,TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

ModuleNotFoundError: ignored

In [68]:
tokenizer=AutoTokenizer.from_pretrained('skt/kogpt2-base-v2',
                                        bos_token='</s>',eos_token='</s>',pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
#from transformers import PreTrainedTokenizerFast,GPT2ForSequenceClassification

#tokenizer=PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2',
#                                                bos_token='</s>',eos_token='</s>',unk_token='<unk>',
#                                                 pad_token='<pad>',mask_token='<mask>')

In [69]:
from soynlp.normalizer import *
import re

def normalization(df):
  clean=[]
  for text in df.reviews.to_list():

    text = re.sub('\.*!+', '!', text)
    text = re.sub('\.*\?+', '?', text)
    text = re.sub('\.+', '.', text)
    text = re.sub(',+', ',', text)
    text = re.sub('[^A-Za-z0-9가-힣\s\[\].,!?\"\']', '', text)
    text = emoticon_normalize(only_text(text), num_repeats=1)
    text = re.sub('\s+', ' ', text)
    clean.append(text)

  return clean


clean_train=normalization(train)
train['clean_train']=clean_train
print(train.columns)

clean_test=normalization(test)
test['clean_test']=clean_test
print(test.columns)

Index(['reviews', 'target', 'clean_train'], dtype='object')
Index(['reviews', 'clean_test'], dtype='object')


In [70]:
train.head()

,reviews,target,clean_train
0,조아요 처음구입 싸게햇어요,2,조아요 처음구입 싸게햇어요
1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데 25천원가량 주고 사기 너무 아깝네요
2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...
3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...
4,튼튼하고 손목을 잘 받쳐주네요~,5,튼튼하고 손목을 잘 받쳐주네요


In [71]:
test.head()

,reviews,clean_test
0,채소가 약간 시들어 있어요,채소가 약간 시들어 있어요
1,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2,부들부들 좋네요 입어보고 시원하면 또 살게요,부들부들 좋네요 입어보고 시원하면 또 살게요
3,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요


In [72]:
train['target'].value_counts()

5    10000
2     8000
1     4500
4     2500
Name: target, dtype: int64

In [12]:
max=0
for i in range(len(train)):
  if max<len(train['clean_train'][i]):
    max=len(train['clean_train'][i])

print(max)

140


In [13]:
max=0
for i in range(len(test)):
  if max<len(test['clean_test'][i]):
    max=len(test['clean_test'][i])

print(max)

140


In [73]:
max_seq_len=128

In [74]:
print(tokenizer.encode('부들부들 좋네요 입어보고 시원하면 또 살게요'))

[9064, 7285, 7669, 7285, 9677, 7098, 8084, 34385, 11213, 24257, 9511, 9108, 27769, 8084]


In [75]:
print(tokenizer.tokenize('부들부들 좋네요 입어보고 시원하면 또 살게요'))

['▁부', '들', '부', '들', '▁좋', '네', '요', '▁입어', '보고', '▁시원', '하면', '▁또', '▁살게', '요']


In [76]:
tokenizer.decode(tokenizer.encode('부들부들 좋네요 입어보고 시원하면 또 살게요'))

'부들부들 좋네요 입어보고 시원하면 또 살게요'

In [77]:
def convert_examples_to_features(examples,labels,max_seq_len,tokenizer):
  input_ids,data_labels=[],[]

  for example,label in tqdm(zip(examples,labels),total=len(examples)):
    bos_token=[tokenizer.bos_token]
    eos_token=[tokenizer.eos_token]

    tokens=bos_token+tokenizer.tokenize(example)+eos_token

    input_id=tokenizer.convert_tokens_to_ids(tokens)
    input_id=pad_sequences([input_id],maxlen=max_seq_len,value=tokenizer.pad_token_id,padding='post')[0]

    assert len(input_id)==max_seq_len, "Error with input length {} vs {}".format(len(input_id),max_seq_len)
    input_ids.append(input_id)
    data_labels.append(label)

  input_ids=np.array(input_ids,dtype=int)
  data_labels=np.asarray(data_labels,dtype=np.int32)

  return input_ids,data_labels


In [80]:
from sklearn.model_selection import train_test_split

ttrain,ttest=train_test_split(train,test_size=0.2,random_state=0)

In [81]:
print(ttrain.shape)
print(ttest.shape)

(20000, 3)
(5000, 3)


In [84]:
#토큰화
X_train,y_train=convert_examples_to_features(ttrain['clean_train'],ttrain['target'],max_seq_len=max_seq_len,
                                             tokenizer=tokenizer)

100%|██████████| 20000/20000 [00:04<00:00, 4618.53it/s]


In [85]:
X_test,y_test=convert_examples_to_features(ttest['clean_train'],ttest['target'],max_seq_len=max_seq_len,
                                           tokenizer=tokenizer)

100%|██████████| 5000/5000 [00:01<00:00, 4542.14it/s]


In [87]:
print('단어에 대한 정수 인코딩 :',X_train[3])
print('각 인코딩의 길이 : ',len(X_train[3]))
print('정수 인코딩 복원 : ',tokenizer.decode(X_train[3]))
print('레이블 :',y_train[3])

단어에 대한 정수 인코딩 : [    1  9209 16623 12248 12594  9193 27018 44361  7312  9497  9713 47322
 20437  9183  9205 26818 21488  9427 19718  9138  8676 10309 13454  9306
  6860 10056  9200  9713  9356 10247 12726 38306  9069  7208  9329 25856
 11690 13935  9021 11562  7235 23410 10973     1     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 :  128
정수 인코딩 복원 :  </s> 사용설명서도 없고 건전지 넣을때 앞쪽으로 넣어서 불이 안들어와서 근처에서 다시 사서 반품신청했는데 다른것 보고 뒤쪽으로 열어서 넣어보니 되더라고요. 2개 구입으로 가격도 2배 되고</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [89]:
model=TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.2.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [90]:
input_ids_layer=tf.keras.layers.Input(shape=(max_seq_len,),dtype=tf.int32)
outputs=model([input_ids_layer])

In [58]:
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 140, 768), dtype=tf.float32, name=None), name='tfgpt2_model/transformer/Reshape_2:0', description="created by layer 'tfgpt2_model'")


In [57]:
print(outputs[0][:, -1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")


In [100]:
#분류 모델 클래스 만들기

class TFGPT2ForSequenceClassification(tf.keras.Model):
  def __init__(self,model_name):
    super(TFGPT2ForSequenceClassification,self).__init__()
    self.gpt=TFGPT2Model.from_pretrained(model_name,from_pt=True)
    self.dropout=tf.keras.layers.Dropout(0.2)
    self.classifier=tf.keras.layers.Dense(1,
                                          kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                          activation='softmax',
                                          name='classifier')
  def call(self,inputs):
    outputs=self.gpt(input_ids=inputs)
    cls_token=outputs[0][:,-1]
    cls_token=self.dropout(cls_token)
    prediction=self.classifier(cls_token)

    return prediction
    

In [92]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.49.133.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.49.133.138:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [93]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [52]:
train['target'].value_counts()

5    10000
2     8000
1     4500
4     2500
Name: target, dtype: int64

In [101]:
with strategy.scope():
  model=TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
  loss=tf.keras.losses.CategoricalCrossentropy()
  model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.2.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [42]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [102]:
'''early_stopping=EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=2
)'''

model.fit(X_train,y_train,epochs=2,batch_size=16,validation_data=(X_test,y_test))

Epoch 1/2
1250/1250 [==============================] - 154s 74ms/step - loss: 0.0000e+00 - accuracy: 0.1785 - val_loss: 0.0000e+00 - val_accuracy: 0.1858
Epoch 2/2
1250/1250 [==============================] - 87s 70ms/step - loss: 0.0000e+00 - accuracy: 0.1785 - val_loss: 0.0000e+00 - val_accuracy: 0.1858


In [103]:
results=model.evaluate(X_test,y_test,batch_size=1024)
print('test loss,test acc:',results)

5/5 [==============================] - 16s 2s/step - loss: 0.0000e+00 - accuracy: 0.1858
test loss,test acc: [0.0, 0.1858000010251999]


In [35]:
results=model.evaluate(X_test,y_test,batch_size=1024)
print('test loss,test acc:',results)

5/5 [==============================] - 20s 2s/step - loss: 0.0000e+00 - accuracy: 0.1858
test loss,test acc: [0.0, 0.1858000010251999]
